In [10]:
import numpy as np
import pandas as pd
import os
import re

In [66]:
data=pd.read_csv(r"C:\Users\Challa Karthik\Desktop\internship\may 13th 2019\hello.csv") 
data.columns=['google','facebook','amazon','apple','netflix','instagram']
type(data)
data.head(5)

,google,facebook,amazon,apple,netflix,instagram
0,Review_1,hotel_101,hotel_1608,16-Nov-07,Nice Marriot View of my king bed room,good
1,Review_2,hotel_101,hotel_6939,30-Oct-07,"Good hotel, charges for internet access The Ma...",good
2,Review_3,hotel_101,hotel_3976,12-Oct-07,Small but adequate rooms If you have an early ...,good
3,Review_4,hotel_101,hotel_2851,31-Aug-07,"Better than average, some noisy rooms I have s...",good
4,Review_5,hotel_101,hotel_7897,18-Jul-07,Ordinary Although it is highly rated in these ...,bad


In [67]:
netflix_data = data['netflix']
type(data)

pandas.core.frame.DataFrame

In [68]:
netflix_data1 = netflix_data.tolist()
for i in range(len(netflix_data1)):
    netflix_data1[i]=re.sub(r'\s+', ' ', netflix_data[i])
    netflix_data1[i]=re.sub('[\d]', '',netflix_data1[i])

In [69]:
netflix_data[8]

'fine choice for seatac The lobby is attractive but a bit small and noisy. The room itself seemed newly redecorated/remodeled. It was not too big but had a very comfortable king bed with lovely duvet and many pillows. The bathroom was luxury level with granite countertops, attractive large framed mirror. There is a free computer off the lobby for printing boarding passes. The Hilton is a block away so you can easily walk over for their restaurant. It is a large steak house type place but there are other choices as well. As noted here, the parking is a ripoff. We dropped off our rental car early to avoid the overnight charge.'

In [70]:
netflix_data1[10]

'Very nice but expensive parking This hotel is very nicely renovated with a modern bar and beautiful indoor pool in the treed atrium. The staff was very friendly and helpful. The rooms are immaculately clean and the beds are extremely comfortable due to the nice linens and down pillows. My only concern was the expensive parking. It will run you $ per day which I expect in L.A., but not Seattle. Other than that we had a wonderful stay and good food in the restaurant.'

In [71]:
#splitting the data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(netflix_data1,data['instagram'],test_size=0.3,random_state=1234)

In [72]:
y_train.value_counts()

good         2225
excellent    1384
bad           791
Name: instagram, dtype: int64

In [73]:
y_test.value_counts()

good         988
excellent    587
bad          312
Name: instagram, dtype: int64

In [74]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [75]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,2),stop_words='english')
X_train_tfidf = tfidf_transformer.fit_transform(X_train)

In [76]:
X_train_tfidf.shape

(4400, 215338)

In [77]:
X_test_tfidf = tfidf_transformer.transform(X_test)
X_test_tfidf.shape

(1887, 215338)

In [78]:
from sklearn.naive_bayes import MultinomialNB

In [79]:
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train)
pred_train = nb_clf.predict(X_train_tfidf)

In [80]:
from sklearn.metrics import confusion_matrix
prediction = confusion_matrix(y_train, pred_train)

In [81]:
#metrics
print("Train_confusion Matirx:\n", prediction,"\n")

Train_confusion Matirx:
 [[   6    0  785]
 [   0  200 1184]
 [   0    0 2225]] 



In [82]:
from sklearn.metrics import recall_score, precision_score, accuracy_score

In [83]:
#Metrics on the trian data
print("Train_Confusion Matrix:\n",prediction,"\n")

from sklearn.metrics import recall_score, precision_score, accuracy_score

acc = accuracy_score(y_train, pred_train)
rec = recall_score(y_train, pred_train, average='micro')
prec = precision_score(y_train, pred_train, average= 'micro')

print("Results of Naive Bayes on train data:","\nAccuracy:",acc, "\nRecall:",rec, "\nPrecision:",prec)

Train_Confusion Matrix:
 [[   6    0  785]
 [   0  200 1184]
 [   0    0 2225]] 

Results of Naive Bayes on train data: 
Accuracy: 0.5525 
Recall: 0.5525 
Precision: 0.5525


In [84]:
#prediction on the test data
pred = nb_clf.predict(X_test_tfidf)
prediction = confusion_matrix(y_test,pred)

In [85]:
print("Test_Confusion matrix:\n", prediction,"\n")
acc = accuracy_score(y_test, pred)
rec = recall_score(y_test,pred, average='micro')
prec = precision_score(y_test, pred,average='micro')

Test_Confusion matrix:
 [[  0   0 312]
 [  0   3 584]
 [  0   1 987]] 



In [86]:
print("Results of Naive Bayes on the test data:", "\nAcc: ",acc,"\nRec:", rec, "\nPrec:", prec)

Results of Naive Bayes on the test data: 
Acc:  0.5246422893481717 
Rec: 0.5246422893481717 
Prec: 0.5246422893481717


In [87]:
#logistic regrssion
from sklearn import linear_model
logreg_train = linear_model.LogisticRegression()
lr_clf = logreg_train.fit(X_train_tfidf, y_train)
pred_lr_train=lr_clf.predict(X_train_tfidf)
pred_lr_test=lr_clf.predict(X_test_tfidf)



c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [88]:
print("Train_Confusion matrix:\n", confusion_matrix(y_train,pred_lr_train))
print("Test_Confusion matrix:\n", confusion_matrix(y_test,pred_lr_test))

Train_Confusion matrix:
 [[ 601    1  189]
 [   0 1103  281]
 [   3   22 2200]]
Test_Confusion matrix:
 [[115   3 194]
 [  0 264 323]
 [ 12 104 872]]


In [89]:
print("Results of logistic regression on test data")
acc = accuracy_score(y_test,pred_lr_test)
rec=recall_score(y_test,pred, average='micro')
prec=precision_score(y_test,pred, average='micro')
print("\nAcc: ",acc,"\nRec:", rec, "\nPrec:", prec)

Results of logistic regression on test data

Acc:  0.6629570747217806 
Rec: 0.5246422893481717 
Prec: 0.5246422893481717


In [90]:
#fine tune the model using grid search
from sklearn.model_selection import GridSearchCV

In [91]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [92]:
grid_values = {'instagram': ['11', '12'], 'google':[0.0001,0.002,0.0222]}

In [93]:
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values, scoring = 'recall')

In [94]:
import numpy as np


In [95]:
from sklearn import linear_model, datasets

In [96]:
from sklearn.model_selection import GridSearchCV

In [97]:
iris = datasets.load_iris()

In [98]:
X = iris.data

In [99]:
y = iris.target

In [100]:
#creating a logistic regression

In [101]:
logistic = linear_model.LogisticRegression()

In [102]:
#create  a regularization penalty space

In [103]:
penalty = ['l1', 'l2']

In [104]:
C = np.logspace(0,4,10)

In [105]:
hyperparameters = dict(C=C, penalty=penalty)

In [106]:
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

In [107]:
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV


In [108]:
# Load data
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [109]:
# Create logistic regression
logistic = linear_model.LogisticRegression()

In [110]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [111]:
# Create grid search using 5-fold cross validation
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

In [112]:
# Fit grid search
best_model = clf.fit(X, y)

c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence th

c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence th

c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence th

c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence th

c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\challa karthik\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence th

In [113]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 7.742636826811269


In [114]:
# Predict target vector
best_model.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [115]:
from sklearn.feature_extraction.text import CountVectorizer

In [117]:
count_vect = CountVectorizer()

In [123]:
X_train_counts = count_vect.fit_transform(netflix_data)
X_train_counts.shape

(6287, 21560)

In [124]:
X_train_counts.shape

(6287, 21560)

In [128]:
count_vect.vocabulary_.get(u'algorithm')

In [129]:
from sklearn.naive_bayes import MultinomialNB


In [130]:
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train)
pred_train = nb_clf.predict(X_train_tfidf)

In [131]:
from sklearn.metrics import confusion_matrix
prediction = confusion_matrix(y_train, pred_train)

In [132]:
#metrics
print("Train_confusion Matirx:\n", prediction,"\n")

Train_confusion Matirx:
 [[   6    0  785]
 [   0  200 1184]
 [   0    0 2225]] 



In [133]:
from sklearn.metrics import recall_score, precision_score, accuracy_score

In [134]:
#Metrics on the trian data
print("Train_Confusion Matrix:\n",prediction,"\n")

from sklearn.metrics import recall_score, precision_score, accuracy_score

acc = accuracy_score(y_train, pred_train)
rec = recall_score(y_train, pred_train, average='micro')
prec = precision_score(y_train, pred_train, average= 'micro')

print("Results of Naive Bayes on train data:","\nAccuracy:",acc, "\nRecall:",rec, "\nPrecision:",prec)

Train_Confusion Matrix:
 [[   6    0  785]
 [   0  200 1184]
 [   0    0 2225]] 

Results of Naive Bayes on train data: 
Accuracy: 0.5525 
Recall: 0.5525 
Precision: 0.5525


In [135]:
#prediction on the test data
pred = nb_clf.predict(X_test_tfidf)
prediction = confusion_matrix(y_test,pred)

In [136]:
print("Test_Confusion matrix:\n", prediction,"\n")
acc = accuracy_score(y_test, pred)
rec = recall_score(y_test,pred, average='micro')
prec = precision_score(y_test, pred,average='micro')

Test_Confusion matrix:
 [[  0   0 312]
 [  0   3 584]
 [  0   1 987]] 



In [137]:
print("Results of Naive Bayes on the test data:", "\nAcc: ",acc,"\nRec:", rec, "\nPrec:", prec)

Results of Naive Bayes on the test data: 
Acc:  0.5246422893481717 
Rec: 0.5246422893481717 
Prec: 0.5246422893481717
